In [1]:
using LinearAlgebra
using Printf
include("../Tensor.jl")

updateLeft (generic function with 1 method)

In [2]:
# initializing |GHZN> state
N = 10
GHZN = zeros(2^N,1);
GHZN[1] = 1/sqrt(2);
GHZN[end] = 1/sqrt(2);

## MPS form of GHZN state

In [3]:
# my code
M = reshape(GHZN*sqrt(2),(2,2^(N-1))); # first site
GHZ_mps = Array{Any}(undef,1,N);

for itN in 1:N
    if itN < N
        U,S,Vt,dw = svdTr(M,2,[1],[],[])
        if itN == 1
            GHZ_mps[itN] = reshape(U,(1,2,2))
        else
            GHZ_mps[itN] = reshape(U,(2,2,2))
        end
        M = contract(diagm(S),2,2,Vt,2,1)
        M = reshape(M,2^2,2^(N-(itN+1)))
    else
        GHZ_mps[itN] = reshape(M,(2,2,1))
    end
end

In [4]:
# verification

M_N  = GHZ_mps[N]/sqrt(2) # left canonical

for itN in 1:N
    if itN < N
        MM = contract(GHZ_mps[itN],3,[1,2],conj(GHZ_mps[itN]),3,[1,2])
    else
        MM = contract(M_N,3,[1,2],conj(M_N),3,[1,2])
    end
    Id = I(size(MM,1))
    err = norm(MM-Id)
    @printf("error %d : %.4e\n", itN, err)
end

error 1 : 0.0000e+00
error 2 : 0.0000e+00
error 3 : 0.0000e+00
error 4 : 0.0000e+00
error 5 : 0.0000e+00
error 6 : 0.0000e+00
error 7 : 0.0000e+00
error 8 : 0.0000e+00
error 9 : 0.0000e+00
error 10 : 2.2204e-16


In [5]:
M_1 = GHZ_mps[1]/sqrt(2) # right canonical

for itN in 1:N
    if itN > 1
        MM = contract(GHZ_mps[itN],3,[2,3],conj(GHZ_mps[itN]),3,[2,3])
    else
        MM = contract(M_1,3,[2,3],conj(M_1),3,[2,3])
    end
    Id = I(size(MM,1))
    err = norm(MM-Id)
    @printf("error %d : %.4e\n", itN, err)
end

error 1 : 2.2204e-16
error 2 : 0.0000e+00
error 3 : 0.0000e+00
error 4 : 0.0000e+00
error 5 : 0.0000e+00
error 6 : 0.0000e+00
error 7 : 0.0000e+00
error 8 : 0.0000e+00
error 9 : 0.0000e+00
error 10 : 0.0000e+00


### Effect of some discarding singular values

In [6]:
Nkeep = 2 # keeping only 2 largest singular values
dw_set = zeros(N-1,1)

M = reshape(GHZN*sqrt(2),(2,2^(N-1))); # first site
GHZ_mps = Array{Any}(undef,1,N);

for itN in 1:N
    if itN < N
        U,S,Vt,dw = svdTr(M,2,[1],Nkeep,[])
        dw_set[itN] = dw
        @printf("discarded weight of the SVD on site %d: %f\n",itN ,dw_set[itN])
        if itN == 1
            GHZ_mps[itN] = reshape(U,(1,2,2))
        else
            GHZ_mps[itN] = reshape(U,(2,2,2))
        end
        M = contract(diagm(S),2,2,Vt,2,1)
        M = reshape(M,2^2,2^(N-(itN+1)))
    else
        GHZ_mps[itN] = reshape(M,(2,2,1))
    end
end

discarded weight of the SVD on site 1: 0.000000
discarded weight of the SVD on site 2: 0.000000
discarded weight of the SVD on site 3: 0.000000
discarded weight of the SVD on site 4: 0.000000
discarded weight of the SVD on site 5: 0.000000
discarded weight of the SVD on site 6: 0.000000
discarded weight of the SVD on site 7: 0.000000
discarded weight of the SVD on site 8: 0.000000
discarded weight of the SVD on site 9: 0.000000


Bond dimension D for GHZ states = 2, thus the discarderd weights = 0 for every site if we are keeping two singular values.

In [7]:
Nkeep = 1 # keeping only 2 largest singular values
dw_set = zeros(N-1,1)

M = reshape(GHZN*sqrt(2),(2,2^(N-1))); # first site
GHZ_mps = Array{Any}(undef,1,N);

for itN in 1:N
    if itN < N
        U,S,Vt,dw = svdTr(M,2,[1],Nkeep,[])
        dw_set[itN] = dw
        @printf("discarded weight of the SVD on site %d: %f\n",itN ,dw_set[itN])
        if itN == 1
            GHZ_mps[itN] = reshape(U,(1,2,2))
        else
            GHZ_mps[itN] = reshape(U,(2,2,2))
        end
        M = contract(diagm(S),2,2,Vt,2,1)
        M = reshape(M,2^2,2^(N-(itN+1)))
    else
        GHZ_mps[itN] = reshape(M,(2,2,1))
    end
end

discarded weight of the SVD on site 1: 1.000000


DimensionMismatch: DimensionMismatch: new dimensions (1, 2, 2) must be consistent with array size 2

The code breaks when we use Nkeep. It is because, we are hardcoding the MPS tensor dimension. In order to truncate, we need to change the dimension of MPS tensor according to the bond dimension. This can be done by modifying the code in the following manner.

In [8]:
## Generalize code for MPS creation

M = reshape(GHZN*sqrt(2),(1,2,2^(N-1))); # first site
GHZ_mps = Array{Any}(undef,1,N);
for itN in 1:N-1
    global M
    U,S,Vt,dw = svdTr(M,3,[1,2],[],[])
    GHZ_mps[itN] = U
    M = contract(diagm(S),2,2,Vt,2,1)
    M = reshape(M,(size(M,1),2,Int32(size(M,2)/2))) # dividing by 2 since we are reshaping 
    # we are splitting both virtual and physical indices.
end
GHZ_mps[end] = M

2×2×1 Array{Float64, 3}:
[:, :, 1] =
 1.0   0.0
 0.0  -1.0

### Checking the case of Nkeep=1

In [9]:
# Nkeep = 1
Nkeep = 1 # keeping only 2 largest singular values
dw_set = zeros(N-1,1)

M = reshape(GHZN*sqrt(2),(1,2,2^(N-1))); # first site
GHZ_mps = Array{Any}(undef,1,N);
for itN in 1:N-1
    global M
    U,S,Vt,dw = svdTr(M,3,[1,2],Nkeep,[]);
    dw_set[itN] = dw;
    @printf("discarded weight of the SVD on site %d: %f\n",itN ,dw_set[itN]);
    GHZ_mps[itN] = U;
    M = contract(diagm(S),2,2,Vt,2,1);
    M = reshape(M,(size(M,1),2,Int32(size(M,2)/2))); # dividing by 2 since we are reshaping 
    # we are splitting both virtual and physical indices.
end
GHZ_mps[end] = M;




discarded weight of the SVD on site 1: 1.000000
discarded weight of the SVD on site 2: 0.000000
discarded weight of the SVD on site 3: 0.000000
discarded weight of the SVD on site 4: 0.000000
discarded weight of the SVD on site 5: 0.000000
discarded weight of the SVD on site 6: 0.000000
discarded weight of the SVD on site 7: 0.000000
discarded weight of the SVD on site 8: 0.000000
discarded weight of the SVD on site 9: 0.000000


1×2×1 Array{Float64, 3}:
[:, :, 1] =
 1.0  0.0

for Nkeep=1, some discarded weight(s) are nonzero.